In [9]:
from func.ultis import compressed_pickle
import matplotlib.pyplot as plt

from func.run_pipeline_super_vox import reassign
import numpy as np
from func.plot_utils import colorful_seg, fill_noise

In [10]:
from os import listdir

def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

hand_seg_list = []
raw_img_list = []
boundary_list = []


test_file_names = [f for f in listdir("../data/CellSeg_dataset/Ovules_processed_thin_boundary_edge/test/")]
test_file_names.remove(".DS_Store")
for i, test_file in enumerate(test_file_names):
    hf = np.load(f"../data/CellSeg_dataset/Ovules_processed_thin_boundary_edge/test/{test_file}")
    raw_img = normalize_data(np.array(hf["raw"], dtype=np.float))
    hand_seg = np.array(hf["ins"], dtype=np.float)
    boundary = np.array(hf["boundary"], dtype=np.float)

    hand_seg_list.append(hand_seg)
    boundary_list.append(boundary)
    raw_img_list.append(raw_img)


train_file_names = [f for f in listdir("../data/CellSeg_dataset/Ovules_processed_thin_boundary_edge/train/")]
train_file_names.remove(".DS_Store")
for i, train_file in enumerate(train_file_names):
    hf = np.load(f"../data/CellSeg_dataset/Ovules_processed_thin_boundary_edge/train/{train_file}")
    raw_img = normalize_data(np.array(hf["raw"], dtype=np.float))
    hand_seg = np.array(hf["ins"], dtype=np.float)
    boundary = np.array(hf["boundary"], dtype=np.float)

    hand_seg_list.append(hand_seg)
    boundary_list.append(boundary)
    raw_img_list.append(raw_img)

/var/folders/qq/9q6wphrj19z8xt4qcffm8df80000gn/T/ipykernel_96702/1575281564.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_img = normalize_data(np.array(hf["raw"], dtype=np.float))
/var/folders/qq/9q6wphrj19z8xt4qcffm8df80000gn/T/ipykernel_96702/1575281564.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  hand_seg = np.array(

# Calculate SSIM

In [17]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm.notebook import tqdm

ssim_list = []
psnr_list = []

for i in tqdm(range(len(raw_img_list))):
    ssim_value = ssim(raw_img_list[i], boundary_list[i], channel_axis=None)
    print(ssim_value)
    ssim_list.append(ssim_value)
    psnr_list.append(psnr(raw_img_list[i], boundary_list[i], data_range=1))

  0%|          | 0/31 [00:00<?, ?it/s]

0.6998434798628758
0.6939576628354216
0.6738888604017028
0.7791962348096091
0.6809081711570757
0.4909198262072595
0.34663486207611266
0.670258597987072
0.7530464351681565
0.7438313526417133
0.678537385614125
0.8155377998604625
0.3995136168420515
0.6718017981364455
0.606572794350784
0.7407292881283303
0.7144309038680539
0.7447374010563745
0.7583113451499399
0.8769017994585053
0.748154952469978
0.8044125116312437
0.7628888236148407
0.7139262697693377
0.6787037402033399
0.706878431064838
0.5205579483878751
0.7354410164398552
0.8261367304102869
0.8685207721170467
0.7435590171797632


In [18]:
# save it
compressed_pickle("ssim_list_Ovules", ssim_list)
compressed_pickle("psnr_list_Ovules", psnr_list)

# Calculate the number of cell instances and their size

In [14]:
value_count_list = []
for i in tqdm(range(len(hand_seg_list))):
    value_count_list.append(np.unique(hand_seg_list[i], return_counts=True))

  0%|          | 0/31 [00:00<?, ?it/s]

In [16]:
# save it
compressed_pickle("value_count_list_Ovules", value_count_list)

In [19]:
# background percentage:
background_percentages_list = []

for hand_seg in hand_seg_list:
    background_size = np.count_nonzero(hand_seg==0.)
    total_size = hand_seg.size
    background_percentages_list.append(background_size/total_size)